# Defensa de la Arquitectura Cognitiva: Agente Conversacional "VuelaConNosotros"

A continuación, presento la defensa de la prueba de concepto (PoC) desarrollada para el agente conversacional "VuelaConNosotros", abordando la arquitectura, el proceso de implementación y las instrucciones para su validación técnica, en respuesta a la sección 2 del desafío propuesto.

## 1. Proceso de Desarrollo y Enfoque

Para abordar este desafío, adopté un proceso de desarrollo iterativo centrado en la robustez y la escalabilidad. La meta principal fue construir una base sólida para un agente conversacional que no solo cumpliera con los requisitos iniciales, sino que también pudiera crecer en complejidad y funcionalidades de manera ordenada.

Mi enfoque se basó en los siguientes principios:

*   **Diseño Modular (SOLID):** Desde el inicio, estructuré el código siguiendo los principios SOLID. Cada componente (clasificación de intención, orquestación, agentes especializados, herramientas) tiene una única responsabilidad y está desacoplado de los demás. Esto facilita las pruebas unitarias, el mantenimiento y la futura expansión del sistema.
*   **Orquestación Explícita:** En lugar de una lógica monolítica, opté por un grafo de estados explícito utilizando LangGraph. Esto permite visualizar y gestionar el flujo de la conversación de manera clara, haciendo que el comportamiento del agente sea predecible y fácil de depurar.
*   **Pruebas Continuas:** Implementé scripts de prueba desde las primeras etapas para validar cada componente de forma aislada y, posteriormente, el flujo conversacional completo. Esto garantizó la detección temprana de errores y la correcta integración de las partes.

## 2. Arquitectura Cognitiva Propuesta

La solución se implementa dentro de un monorepositorio Nx, separando el backend del agente de otras posibles aplicaciones. La arquitectura del backend está diseñada para ser modular y escalable. A continuación, se presenta un diagrama que ilustra el flujo de una interacción:

```mermaid
graph TD
    subgraph "User Interaction"
        U[👤 User]
    end

    subgraph "API Layer (FastAPI)"
        API[🌐 /vuelaconnosotros/chat]
    end

    subgraph "Cognitive Core"
        O[⚙️ LangGraph Orchestrator]
        NLU[🧠 Intent Classifier<br>(Gemini Flash)]
        Router{Router}
        
        subgraph "Agents"
            A1[✈️ FlightStatusAgent]
            A2[🔄 FlightChangeAgent]
            A3[💬 GeneralAgent]
        end

        subgraph "Tools"
            T1[🛠️ get_flight_status]
            T2[🛠️ get_flight_availability]
        end
    end

    subgraph "Observability"
        LS[📊 LangSmith]
    end

    U --> API
    API --> O
    O --> NLU
    NLU --> Router
    Router -- "consultar_estado_vuelo" --> A1
    Router -- "cambiar_vuelo" --> A2
    Router -- "saludo/despedida" --> A3
    
    A1 --> T1
    A2 --> T2
    
    T1 --> A1
    T2 --> A2

    A1 --> O
    A2 --> O
    A3 --> O

    O --> API
    API --> U

    O --trace--> LS
    NLU --trace--> LS
    A1 --trace--> LS
    A2 --trace--> LS
    A3 --trace--> LS
    T1 --trace--> LS
    T2 --trace--> LS
```


*   **API Layer (`api/`):** Un endpoint FastAPI (`/chat`) que sirve como única puerta de entrada para las interacciones del usuario. Se encarga de recibir los mensajes, gestionar las sesiones y devolver las respuestas del agente.
*   **Orquestador (`orchestrator/`):** El cerebro de la aplicación, implementado con LangGraph. Este componente gestiona el estado de la conversación y dirige el flujo de trabajo. Su ciclo básico es:
    1.  Clasificar la intención del usuario.
    2.  Enrutar la conversación al agente apropiado.
    3.  Ejecutar las herramientas del agente si es necesario.
    4.  Generar una respuesta para el usuario.
    5.  Persistir el estado y esperar la siguiente interacción.
*   **NLU (`nlu/`):** Un módulo de clasificación de intención que utiliza el modelo **Gemini Flash**. Analiza el mensaje del usuario y lo categoriza en una de las intenciones predefinidas (ej. `consultar_estado_vuelo`, `cambiar_vuelo`, `saludo`). Su precisión es clave para enrutar correctamente la solicitud.
*   **Agentes Especializados (`agents/`):** Siguiendo el principio de responsabilidad única, he creado agentes específicos para cada intención principal. Cada agente tiene acceso a un conjunto limitado de herramientas necesarias para su tarea. Por ejemplo:
    *   `FlightStatusAgent`: Equipado con herramientas para verificar el estado de un vuelo.
    *   `FlightChangeAgent`: Orquesta un sub-flujo para recopilar la información necesaria para un cambio de vuelo (vuelo actual, destino, fecha).
*   **Herramientas (`tools/`):** Simulaciones de servicios externos, como la consulta a una base de datos de vuelos o un sistema de reservas. Están diseñadas como funciones simples y atómicas que los agentes pueden invocar.
*   **Observabilidad (LangSmith):** Toda la traza de ejecución, desde la clasificación de la intención hasta la llamada a las herramientas, está integrada con LangSmith. Esto proporciona una visibilidad completa del razonamiento del agente, lo cual es fundamental para la depuración y la mejora continua.

## 3. Instrucciones Técnicas para la Puesta en Marcha

Para evaluar la solución, he preparado un entorno de prueba completo. Los siguientes pasos describen cómo ejecutarlo:

**Requisitos Previos:**

*   Tener Python 3.10+ y `uv` (o `pip`) instalados.
*   Disponer de una clave de API de Google AI y una de LangSmith.

**Pasos para la Ejecución:**

1.  **Configurar el Entorno:**
    *   Navegar al directorio del backend: `cd apps/itti-backend`.
    *   Crear un archivo `.env` a partir del ejemplo `.env.example`.
    *   Rellenar las variables `GOOGLE_API_KEY` y las claves de LangSmith en el archivo `.env`.

2.  **Instalar Dependencias:**
    *   Recomiendo usar `uv` para una instalación rápida: `uv venv` y `uv pip install -r requirements.txt`.

3.  **Iniciar el Servidor:**
    *   Ejecutar el siguiente comando desde el directorio `apps/itti-backend`:
        ```bash
        uvicorn itti_backend.main:app --host 0.0.0.0 --port 8003 --reload
        ```
    *   El servidor estará disponible en `http://localhost:8003`.

4.  **Ejecutar la Demostración:**
    *   En una terminal separada, dentro del mismo directorio, ejecutar el script de demostración avanzado:
        ```bash
        python advanced_demo.py
        ```
    *   Este script simula conversaciones completas, cubriendo todos los flujos implementados (saludo, consulta de estado, cambio de vuelo y manejo de preguntas fuera de tópico). También ejecuta pruebas de casos borde para validar la robustez del agente.

5.  **Verificar Trazas:**
    *   Acceder a la URL de LangSmith proporcionada por la salida del script para inspeccionar las trazas detalladas de cada interacción.

## 4. Resultados y Conclusiones

La prueba de concepto demuestra con éxito la viabilidad de la arquitectura propuesta. Los resultados clave son:

*   **Precisión en la Intención:** El uso de Gemini Flash para la clasificación de intenciones ha demostrado ser altamente efectivo, incluso con entradas ambiguas o complejas.
*   **Robustez del Flujo:** El grafo de LangGraph gestiona la conversación de manera predecible, guiando al usuario a través de procesos de varios pasos (como el cambio de vuelo) y manejando adecuadamente las interrupciones o cambios de contexto.
*   **Modularidad y Escalabilidad:** La arquitectura actual permite añadir nuevas funcionalidades de forma sencilla. Por ejemplo, para gestionar reservas de hoteles, bastaría con crear un nuevo `HotelAgent` con sus propias herramientas y añadir una nueva ruta en el orquestador, sin necesidad de modificar los agentes existentes.

En conclusión, considero que esta PoC establece una base técnica sólida y profesional para el desarrollo de un agente conversacional avanzado, cumpliendo con los más altos estándares de diseño de software y preparado para futuras iteraciones.